In [ ]:
from oemof import solph

import pickle
import os
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.dates import DateFormatter, YearLocator, MonthLocator
import matplotlib
import oemof_visio as oev
%matplotlib inline
matplotlib.rcParams['figure.figsize'] = [20.0, 7.0]

### Restore the whole energysystem with results

In [ ]:
energysystem = solph.EnergySystem()
energysystem.restore(dpath=None, filename=None)

### Convert keys to strings and print all keys

In [ ]:
string_results = solph.views.convert_keys_to_strings(energysystem.results['main'])
print(string_results.keys())

### Use the outputlib to collect all the flows into and out of the electricity bus
Collect all flows into and out of the electricity bus by using solph.views.node()

In [ ]:
node_results_bel = solph.views.node(energysystem.results['main'], 'bel')

What we will be working with now is a pandas dataframe. Have a look at these links to learn about pandas, especially the last one (pandas in 10min):  
https://pandas.pydata.org/  
http://pandas.pydata.org/pandas-docs/stable/  
http://pandas.pydata.org/pandas-docs/stable/10min.html

In [ ]:
df = node_results_bel['sequences']
df.head(2)

### Use pandas functionality to create a plot of all the columns of the dataframe

In [ ]:
ax = df.plot(kind='line', drawstyle='steps-post')
ax.set_xlabel('Time [h]')
ax.set_ylabel('Energy [MWh]')
ax.set_title('Flows into and out of bel')
ax.legend(loc='center left', bbox_to_anchor=(1.0, 0.5)) # place legend outside of plot
plt.show()

### oemof visio provides a function that collects the column names for in and outflows as lists in a dictionary

In [ ]:
in_out_dictionary = oev.plot.divide_bus_columns('bel', df.columns)
in_cols = in_out_dictionary['in_cols']
out_cols = in_out_dictionary['out_cols']

This allows us to get the all the columns that are outflows:

We can get any column of the dataframe by providing its label as a list

In [ ]:
bel_to_demand_el = [(('bel', 'demand_el'), 'flow')] #  this is a list with one element
df[bel_to_demand_el].head(2)

### Plot only outflows

In [ ]:
ax = df[out_cols].plot(kind='line', drawstyle='steps-post')
ax.set_xlabel('Time [h]')
ax.set_ylabel('Energy [MWh]')
ax.set_title('Flows into or out of bel')
ax.legend(loc='center left', bbox_to_anchor=(1.0, 0.5)) # place legend outside of plot
plt.show()

### Use the functions of oemof_visio to create plots
See also: oemof_examples/examples/oemof_0.2/plotting_examples/storage_investment_plot.py


Use color palette generators to generate a suitable color list, e.g.:     
http://javier.xyz/cohesive-colors/  
https://colourco.de/  
http://seaborn.pydata.org/tutorial/color_palettes.html  

In [ ]:
inorder = [(('pp_chp', 'bel'), 'flow'),
             (('pp_coal', 'bel'), 'flow'),
             (('pp_gas', 'bel'), 'flow'),
             (('pp_lig', 'bel'), 'flow'),
             (('pp_oil', 'bel'), 'flow'),
             (('pv', 'bel'), 'flow'),
             (('wind', 'bel'), 'flow')]

outorder = [(('bel', 'demand_el'), 'flow'),
             (('bel', 'excess_el'), 'flow'),
             (('bel', 'heat_pump'), 'flow')]

cdict = {(('pp_chp', 'bel'), 'flow'): '#eeac7e',
        (('pp_coal', 'bel'), 'flow'): '#0f2e2e',
        (('pp_gas', 'bel'), 'flow'): '#c76c56',
        (('pp_lig', 'bel'), 'flow'): '#56201d',
        (('pp_oil', 'bel'), 'flow'): '#494a19',
        (('pv', 'bel'), 'flow'): '#ffde32',
        (('wind', 'bel'), 'flow'): '#4ca7c3',
        (('bel', 'demand_el'), 'flow'): '#ce4aff',
        (('bel', 'excess_el'), 'flow'): '#555555',
        (('bel', 'heat_pump'), 'flow'): '#42c77a'}


fig = plt.figure(figsize=(13, 5))

my_plot = oev.plot.io_plot('bel', df,
                           inorder=inorder,
                           outorder=outorder,
                           cdict=cdict,
                           ax=fig.add_subplot(1, 1, 1),
                           smooth=False)
ax = my_plot['ax']
oev.plot.set_datetime_ticks(ax, df.index, tick_distance=32,
                            date_format='%d-%m-%H', offset=12)

my_plot['ax'].set_ylabel('Power in MW')
my_plot['ax'].set_xlabel('2012')
my_plot['ax'].set_title("Electricity bus")
legend = ax.legend(loc='center left', bbox_to_anchor=(1.0, 0.5)) # place legend outside of plot

# save figure
fig = ax.get_figure()
fig.savefig('myplot.png', bbox_inches='tight')